## some useful helper functions to consider
Convert a given string to {current-date}{string}.csv

```
makefn('string')
```

### helpful consolidators
Creates two dataframes of all cages + dispensible % > .2, and either saves them independently or pops them back for  further manip.
```
makeConsolidations(df, save = True, popBack = False)
```    
                  

### helpful for euthanizing many animals
```
df.fillna("-").groupby(['Cage','DOD']).agg({'Tag':'unique', 'Lineage': 'unique'}).to_csv(fn)
```

In [1]:
from ColonyMgmt import * 
import seaborn as sns
import gspread
from tabulate import tabulate
gc = gspread.oauth() #relies on specific json file existing in specific location. not robust, is my point.

In [2]:
sh = gc.open("KalhorLabMice") #this line is liable to break when/if we change the sheetname
ws_list = sh.worksheets()

In [3]:
liveCol = sh.get_worksheet(0) #get first spreadsheet; gsheets uses zero indexing
df = pd.DataFrame(liveCol.get_all_records()) #convert to dataframe
# archived = pd.DataFrame(sh.get_worksheet(1).get_all_records()) #get second sheet and convert to dataframe -- in case of checking historic mice genos etc
# df= df.append(archived) #combine the two

In [4]:
parsePlate?

In [5]:
df['TagClean'] = df.Tag.astype(str)
df['DOB_clean'] = pd.to_datetime(df.DOB)
df = parseDOD(df, drop = True)
if df.Ear.any():
    df.Ear =  df.Ear.apply(lambda row: "\'" + row if type(row)== str else np.nan) #convert ear tags to excel-clean formats
    print("Parsed ear tags for Excel formatting")
colonyStats(df)
df = parsePlate(df)

initial counts: {'numCages': 188, 'numMice': 526} 
after dropping dead {'numCages': 142, 'numMice': 385}
calculted total dispensible counts
calculated total cage sizes
calculated % disp
Parsed ear tags for Excel formatting


In [7]:
df.head()

,Tag,Cage,Ear,Sex,Color,Name,Genotype,DOB,DOD,Father,...,TagClean,DOB_clean,parsedDOD,DispCount,CageSize,DispPercent,PlateID,Row,Column,Position
0,5216,LSL-Syn_201023_c1,'+/t,M,Black,,"fCas9 MT (+/+), synCre (-)",2/27/2020,dispensible,2728,...,5216,2020-02-27,NaT,3.0,3,1.0,T965524,G,09,G9
1,5324,LSL-Syn_201023_c1,'+/t,F,Black,,"fCas9 MT(+/+), synCre (+)",3/24/2020,dispensible,2728,...,5324,2020-03-24,NaT,3.0,3,1.0,T965521,B,04,B4
2,5325,LSL-Syn_201023_c1,'++/t,F,Black,,"fCas9 MT(+/+), synCre (+)",3/24/2020,dispensible,2728,...,5325,2020-03-24,NaT,3.0,3,1.0,T965521,C,04,C4
3,5569,1763PB#7_c4,'+/t,F,Agouti,,"4E,7I,18S,12ina",11/17/2020,dispensible,5444,...,5569,2020-11-17,NaT,2.0,2,1.0,T1129056,H,04,H4
4,5659,1763PB#7_c4,'+/t,F,Black,,"4E,5I,18S,10ina",12/4/2020,dispensible,5239,...,5659,2020-12-04,NaT,2.0,2,1.0,T1103209,E,02,E2


In [8]:
df = calcDisp(df)

calculted total dispensible counts
calculated total cage sizes
calculated % disp


In [9]:
# df.sort_values('DispPercent', ascending = False).head(20)

In [9]:
df = df.sort_values('DOB')

In [12]:
df.query('Lineage.str.contains("PB#7")').groupby("Sex").size()

Sex
F    34
M    29
dtype: int64

In [10]:
df.Lineage.unique()

array(['', '1853PBF#1', 'J024858-Cas9 x PB#21 (not B6)', 'PB#21', 'PBF',
       '1763PB#7', '1763PB#3', 'LSL-Cas9 x SynCre', '1685ins21#2',
       'J024858-Cas9', 'LSL-Cas9 x hCD2', 'C57BL/6J', 'J028555_B6-Cas9',
       'J000664_B6 x PB#21', 'J028555_B6-Cas9 x PB#21'], dtype=object)

In [11]:
# check that lineages are correct
for num, el in enumerate(df.Lineage.unique()):
    print(num, el)

0 
1 1853PBF#1
2 J024858-Cas9 x PB#21 (not B6)
3 PB#21
4 PBF
5 1763PB#7
6 1763PB#3
7 LSL-Cas9 x SynCre
8 1685ins21#2
9 J024858-Cas9
10 LSL-Cas9 x hCD2
11 C57BL/6J
12 J028555_B6-Cas9
13 J000664_B6 x PB#21
14 J028555_B6-Cas9 x PB#21


In [12]:
# remapDict= {'PB3': '1763PB#3',
#             'PB21' : 'PB#21'}
# #             'pb7' : '1763PB#7',
# #             'pbf': '1853PBF#1'}
# for key, value in remapDict.items():
#     df.Lineage = df.Lineage.apply(lambda row: value if key in row else row)

In [13]:
makeConsolidations(df, save = True, popBack = False) #save current state of the colony for quick statuses etc

Saved CSV with fns 211117-AllCages.csv
Saved CSV with fns 211117-FemaleCons.csv


In [14]:
df = df.sort_values('DOB_clean')

In [33]:
grouped = df.query('DOB_clean > "2021-08"').groupby([pd.Grouper(key='DOB_clean',freq='m'), 'Lineage',
            'Sex']).size().unstack('DOB_clean')#.reset_index()
grouped['Total'] = grouped.sum(axis=1)
grouped = grouped.fillna("-")
grouped
# tabulate(grouped, headers='keys', tablefmt='psql')

DOB_clean                         2021-08-31 00:00:00 2021-09-30 00:00:00  \
Lineage                       Sex                                           
1685ins21#2                   F                     -                 2.0   
                              M                     -                 3.0   
1763PB#3                      F                     -                 9.0   
                              M                     -                10.0   
1763PB#7                      F                   3.0                   -   
                              M                   6.0                   -   
J000664_B6 x PB#21            F                  12.0                   -   
                              M                  10.0                   -   
J024858-Cas9 x PB#21 (not B6) F                   5.0                   -   
                              M                   9.0                   -   
J028555_B6-Cas9 x PB#21       F                   8.0                   -   
                              M                   9.0                   -   
PB#21                         F                   5.0                   -   
                              M                   5.0                   -   
PBF                           F                     -                   -   
                              M                     -                   -   

DOB_clean                         2021-10-31 00:00:00  Total  
Lineage                       Sex                             
1685ins21#2                   F                     -    2.0  
                              M                     -    3.0  
1763PB#3                      F                     -    9.0  
                              M                     -   10.0  
1763PB#7                      F                     -    3.0  
                              M                     -    6.0  
J000664_B6 x PB#21            F                     -   12.0  
                              M                     -   10.0  
J024858-Cas9 x PB#21 (not B6) F                     -    5.0  
                              M                     -    9.0  
J028555_B6-Cas9 x PB#21       F                     -    8.0  
                              M                     -    9.0  
PB#21                         F                   1.0    6.0  
                              M                   3.0    8.0  
PBF                           F                   3.0    3.0  
                              M                   1.0    1.0

In [87]:
grouped = df.groupby(['Lineage',
            'Sex',
            pd.Grouper(key='DOB_clean',freq='q')]).size()..reset_index()
grouped['Total'] = grouped.sum(axis=1)
grouped
# grouped.fillna("-").to_csv(saveFn)

,Lineage,Sex,DOB_clean,0,Total
0,1685ins21#2,F,2021-06-30,12,12
1,1685ins21#2,F,2021-09-30,2,2
2,1685ins21#2,M,2021-06-30,6,6
3,1685ins21#2,M,2021-09-30,3,3
4,1763PB#3,F,2020-12-31,4,4
5,1763PB#3,F,2021-06-30,22,22
6,1763PB#3,F,2021-09-30,20,20
7,1763PB#3,M,2021-06-30,22,22
8,1763PB#3,M,2021-09-30,17,17
9,1763PB#7,F,2020-12-31,3,3


In [13]:
df.query('DOD.str.contains("hold") & Sex == "F"')[easyCats + ['DOD','Genotype', 'Father', 'Mother']]

,Tag,Cage,Ear,Sex,Color,DOB,Lineage,Plate,DOD,Genotype,Father,Mother
224,7077,AA0297,'t/++,F,Black,6/18/2021,PB#21,T1103218-H08,hold till 01/01/21,"9E,12I,7S,17ina",5572,5653/5619
275,7033,AA0285,'t+\-,F,Black,7/1/2021,PB#21,T1103218-B03,hold till 01/01/21,"8E,14I,16S,12ina",5483,5642/5671
273,7031,AA0285,'t/+,F,Black,7/1/2021,PB#21,T1103218-H02,hold till 01/01/21,"7E,15I,15S,14ina",5483,5642/5671
271,7029,AA0285,'+/t,F,Agouti,7/1/2021,PB#21,T1103218-F02,hold till 01/01/21,"7E,15I,16S,16ina",5483,5642/5671


In [10]:
df.query('Sex == "F"  \
         & DOB_clean > "2021-06" \
         & ~DOD.str.contains("breeder")').sort_values('DOB_clean',
                                                       ascending=False).groupby([
                                                                                 "Lineage"]).agg({'Tag':'unique',
                                                                                                         'DOB_clean':['min', 'max'], 'Tag':'count', 'DOD':'unique'})
# [easyCats + ['DOD','Genotype', 'Father', 'Mother']]

Tag  DOB_clean             \
                              count        min        max   
Lineage                                                     
1685ins21#2                       2 2021-09-10 2021-09-10   
1763PB#3                         16 2021-06-07 2021-07-22   
1763PB#7                         11 2021-06-18 2021-08-25   
C57BL/6J                          3 2021-06-08 2021-06-08   
J000664_B6 x PB#21               12 2021-08-10 2021-08-17   
J024858-Cas9                      8 2021-06-17 2021-07-10   
J024858-Cas9 x PB#21 (not B6)    11 2021-07-08 2021-08-04   
J028555_B6-Cas9                   3 2021-06-08 2021-06-08   
J028555_B6-Cas9 x PB#21           8 2021-08-10 2021-08-12   
PB#21                             4 2021-06-18 2021-07-01   
PB3                               6 2021-09-24 2021-09-24   

                                                DOD  
                                             unique  
Lineage                                              
1685ins21#2                                      []  
1763PB#3                            [dispensible, ]  
1763PB#7                                         []  
C57BL/6J                                         []  
J000664_B6 x PB#21                               []  
J024858-Cas9                                     []  
J024858-Cas9 x PB#21 (not B6)                    []  
J028555_B6-Cas9                                  []  
J028555_B6-Cas9 x PB#21                          []  
PB#21                          [hold till 01/01/21]  
PB3                                              []

In [12]:
df.query('Sex == "F"  \
         & DOB_clean > "2021-06" \
         & ~DOD.str.contains("breeder") \
        & DOD.str.contains("disp")').sort_values('DOB_clean',
                                                       ascending=False).groupby(['DOD',
                                                                                 "Lineage"]).agg({'Cage':'unique','Tag':['unique', 'count'],
                                                                                                         'DOB_clean':['min', 'max']})
# [easyCats + ['DOD','Genotype', 'Father', 'Mother']]
# 

Cage  \
                                                unique   
DOD         Lineage                                      
dispensible 1763PB#3  [AA0320, AA0326, AA0331, AA0308]   

                                                 Tag        DOB_clean  \
                                              unique count        min   
DOD         Lineage                                                     
dispensible 1763PB#3  [5877, 5891, 7091, 6023, 6025]     5 2021-06-19   

                                 
                            max  
DOD         Lineage              
dispensible 1763PB#3 2021-07-22

In [ ]:
df.DOB.unique()

In [ ]:
df.query('DOD.str.contains("cross")')

In [39]:
df.query('Sex == "M"  \
         & DOB_clean > "2021-06" \
         & ~DOD.str.contains("breeder") \
         & Lineage.str.contains("PB#21")').sort_values('DOB_clean',
                                                       ascending=False)[easyCats + ['DOD','Genotype', 'Father', 'Mother']].groupby(["Sex", "Lineage"])
# .agg({'Tag':'unique','DOB_clean':['min', 'max'], 'Tag':'count'})

,Tag,DOB_clean
count,62.0,NaT
min,NaN,2021-06-16
max,NaN,2021-08-12


In [ ]:
for l in df.Lineage.unique(): print(l)

In [ ]:
df.Lineage.unique()

In [ ]:
# df.query("Lineage.str.contains('21') & DOD.str.contains('disp')").sort_values("DispPercent").groupby(["Lineage", "DOD", "Cage",]).agg({'Tag':'unique', 'Lineage': 'unique', "DispPercent": 'mean'})

In [ ]:
fn = makefn('euthanize_sheet')
df.fillna("-").groupby(['Cage','DOD']).agg({'Tag':'unique', 'Lineage': 'unique'}).to_csv(fn)

In [ ]:
df.DOB[df.DOB.notnull()]

In [ ]:
df.groupby("Lineage").count()

In [ ]:
breeders = (df.DOD == "breeder")

## prolapsed mice 

In [ ]:
# df[breeders].to_clipboard('tsv')

In [ ]:
# #5522 - 2/23/2021
# # 5511 
# # 5693 5/17
# # 5645 5/17
# # 5777 & cagemates ()
# prolapses = [5522, 5511, 5693, 5645, 5777, 5778]
# easyCats.append('DOD')

In [ ]:
# df[df.Tag.isin(prolapses)][easyCats]

In [ ]:
# df.query('Lineage == "PB#21" & DOD.str.contains("back")').Tag.unique()

In [ ]:
df[breeders].sort_values('Cage').groupby(['Cage']).agg({
        'DispPercent': 'mean',
#         'DispCount': 'mean',
        #'Cage': 'size',
        'Sex': 'unique',
        'DOD': 'unique',
        'Tag': 'unique',
        'Color': 'unique',
        'Ear': 'unique',
        'Lineage': 'unique'
    }).to_clipboard()

In [ ]:
mask = ~df.TagClean.isin(tier1) 
experimental = df.Lineage.str.contains("Cas9 x")
PB = df.Lineage.str.contains("PB#")
tier1 = df.DOD.str.contains("tier1_confirmed|breed")
femalesM = (df.Sex.str.lower() == "f")
malesM = (df.Sex.str.lower() == "m")
not_ancient = df.DOB > '2020-04-01'

In [ ]:
cats = ['Tag', 'Ear', 'Sex', 'Color','DOB', 'Lineage']

In [ ]:
claire_january[cats].to_clipboard(index=False, sep="\t")

In [ ]:
def is_number(s):
    try:
        float(s)
        return s
    except ValueError:
        return float(''.join(ele for ele in s if ele.isdigit()))
        print(s)''


In [ ]:
# ts = os.path.getmtime(file)
# print("mtime:", dt.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# df = importCleanDF(file,
#                    dropDeads=False,
#                    dropTransfers=True)

# df = df[~df.DOD.str.contains("euthan|fd")]

In [ ]:
df.Lineage.unique()

In [ ]:
# check that lineages are correct
for num, el in enumerate(df.Lineage.unique()):
    print(num, el)

In [ ]:
# mask = ~df.TagClean.isin(tier1) 
experimental = df.Lineage.str.contains("Cas9 x")
PB = df.Lineage.str.contains("PB#")
tier1 = df.DOD.str.contains("tier1_confirmed|breed")
femalesM = (df.Sex.str.lower() == "f")
malesM = (df.Sex.str.lower() == "m")
not_ancient = df.DOB > '2020-04-01'
ins = df.Lineage.str.contains("ins")
not_new = df.DOB < '2020-10-20'
pb3 = df.Lineage.str.contains("PB#")


In [ ]:
df[ins & not_new][cats+['DOD']].to_clipboard(index=False)


In [ ]:
# a, b = makeConsolidations(df, save = True, popBack = True)

In [ ]:
df.groupby(['Lineage', 'DOD']).agg({'Cage': 'nunique', 'Sex':'unique', 'Tag': ['nunique','unique'], })

In [ ]:
fns = glob.glob(f"{source_dir}/*{date}*/*")

In [ ]:
animal_frames = []

for fn in fns:
        frame = pd.read_csv(fn, sep="\t")
        frame['Lineage'], frame['Date'] = fn[10:24].split("-", 1)
        frame['SourceFn'] = fn
        animal_frames.append(frame)

# speciality functions - check breedings, splitting genotypes

In [ ]:
source_dir = "Breedings"
date = "210827"

breeders = getBreedingList(source_dir, date)
tier2 = set(breeders.query('SourceFn.str.contains("compositions2")')['Tag'])
tier1 = set(breeders.query('SourceFn.str.contains("compositions1")')['Tag'])

# syntax to check if breeder list against colony list
df[df['Tag'].isin(tier1) & df['Tag'].notnull()] 

#if any of the breeding pairs have an NA in them -- i.e. a pair that is set up with only one female, this will match *all* animals that also have a null tag. Check with the below line to see if this is the case.
breeders.query('Tag.isnull()')


In [ ]:
breeders[breeders.Tag.isin(tier2)].sort_values('Tag')

In [ ]:
# df[df['Tag'].isin(tier2) & df['Tag'].notnull()] 

In [ ]:
## TEMPORARY CODE
transfers = df.query('DOD.str.contains("glatzer|west")')
transfers.groupby(['DOD']).agg({'Cage': ['nunique','unique'], 'Tag': ['nunique','unique'], 'Lineage':'unique'}).to_clipboard()

In [ ]:
# to split genotyping info
df = splitGenotypes(df)

In [ ]:
PB21 = df.query("Lineage == 'PB#21'").

In [ ]:
cleaned = PB21.sort_values('E_num', ascending = False).head(10).reset_index()

In [ ]:
trim = cleaned[['Tag', 'E_num', "I_num",'S_num', 'summedhgRNAs_active']].reset_index()
# melt = cleaned.melt(value_vars = ["E_num", "I_num", 
#                                            'S_num', 
#                                            'summedhgRNAs_active'])

In [ ]:
trim

In [ ]:
melt = trim.melt(id_vars = ['index', 'summedhgRNAs_active'], value_vars = ["E_num", "I_num", 
                                           'S_num'], ignore_index = True)
melt['var_percent'] = melt['value']/melt['summedhgRNAs_active']

In [ ]:
melt

In [ ]:
# ax = sns.histplot(df, x='year', hue='country', weights='percentage',
#              multiple='stack', palette='tab20c', shrink=0.8)
ax = sns.histplot(data=melt, x= 'index', hue ="variable", weights = "value", multiple = "stack", bins = 10)
# plt.xticks(range(0,101,10))
legend = ax.get_legend()
legend.set_bbox_to_anchor((1, 1))

In [ ]:
sns.histplot(data = cleaned, x = "")

In [ ]:
import pandas as pd
import seaborn as sns

# Put data in long format in a dataframe.
df = pd.DataFrame({
    'country': countries2012 + countries2013,
    'year': ['2012'] * len(countries2012) + ['2013'] * len(countries2013),
    'percentage': percentage2012 + percentage2013
})

# One liner to create a stacked bar chart.
ax = sns.histplot(df, x='year', hue='country', weights='percentage',
             multiple='stack', palette='tab20c', shrink=0.8)
ax.set_ylabel('percentage')
# Fix the legend so it's not on top of the bars.
legend = ax.get_legend()
legend.set_bbox_to_anchor((1, 1))

In [ ]:
cleaned.max(numeric_only = True)

In [ ]:
cleand.groupby("Lineage")['I_num'].max().dropna()

In [ ]:
sns.displot(PB21, )

In [ ]:
df.groupby(['DOD']).agg({'Cage': 'nunique', 'Tag': ['nunique','unique'], 'Lineage':'unique'})

In [ ]:
mask = ~df.TagClean.isin(tier1) 
experimental = df.Lineage.str.contains("Cas9 x")
PB = df.Lineage.str.contains("PB#")
tier1 = df.DOD.str.contains("tier1_confirmed|breed")
femalesM = (df.Sex.str.lower() == "f")
malesM = (df.Sex.str.lower() == "m")
not_ancient = df.DOB > '2020-04-01'


In [ ]:
df['mutationScore'] = df['Genotype'].str.split(":", expand = True)[1].astype(float)
df['early_and_mid_ct'] = df['E_num'] + df ['I_num']

In [ ]:
df[mask & PB & not_ancient & ~tier1 & malesM][['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'DOB', 'DOD', 'Lineage', 'early_and_mid_ct','summedhgRNAs_active']].sort_values('early_and_mid_ct', ascending = False).to_clipboard(index = False)

In [ ]:
df[mask & experimental & not_ancient & ~tier1][['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'Name', 'Genotype', 'DOB', 'DOD', 'Lineage', 'E_num', 'I_num', 'S_num',
       'ina_num', 'summedhgRNAs', 'summedhgRNAs_active']].sort_values([['E_num', 'I_num'])]
# .to_clipboard(index = False)

In [ ]:
df[mask & experimental & not_ancient & ~tier1][['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'Name', 'Genotype', 'DOB', 'DOD', 'mutationScore']].sort_values('mutationScore', ascending = False).to_clipboard(index = False)

In [ ]:
df.query("Lineage.str.contains('PB')").groupby(["Lineage", 'DOD']).agg({'Tag':'nunique'})

In [ ]:
breedersM = df['Tag'].isin(set(breeders['Tag']))

euthReqM = (df['DOD'] == 'requested euthanasia')
fdM = df.DOD.str.contains("fd")
experimentalM = df.DOD.str.contains("experimental|~breeder") | df.Lineage.str.contains("Exp")
pbM = df.Lineage.str.contains("PB#")
insM = df.Lineage.str.contains("ins")

In [ ]:
working = df[ ~euthReqM & ~fdM].copy().reset_index()

In [ ]:
working['mutationScore'] = working['Genotype'].str.split(":", expand = True)[1].astype(float)

In [ ]:
working[easyCats + ['DOD','Genotype', 'mutationScore', 'Father', 'Mother']].sort_values(['Sex', 'mutationScore'], ascending = False)
# .to_clipboard(index = False)

In [ ]:
working.query("DOD.str.contains('putative')")[easyCats +['DOD']]

In [ ]:
df[malesM & pbM & ~euthReqM][easyCats + ['DOD', 'E_num', 'I_num', 'S_num', 'summedhgRNAs', 'summedhgRNAs_active']].sort_values('summedhgRNAs_active', ascending = False).query('~DOD.str.contains("confirmed|xcas9|transferred|tier1|breeder-may20")')

In [ ]:
meese = df.query(
    'DOD.str.contains("pbf|pb3") & DOD.str.contains("tier1")').sort_values(
        ['Sex', 'Lineage'], ascending=False)
meese.groupby("Cage", sort = False).agg({
    'Tag': 'unique', 'CageSize':'mean',
    'Ear': 'unique',
    'Color': 'unique',
    'Lineage':'unique', 'Sex': 'unique'
})

In [ ]:
meese[easyCats].to_clipboard(index = False)

In [ ]:
# cages_find.head()

In [ ]:
# cages_find.groupby("Cage").agg({'Tag': ['count', 'unique'], 'Ear': 'unique', 'Color': 'unique'}).to_clipboard(sep = "\t")

In [ ]:
# df.query('DOD == "breeding-putative-201015"')[easyCats].sort_values('DOB').to_csv(makefn('PB7_breeders'), sep = "\t")

In [ ]:
# ## vickie cages
# # mutant:
# AA0105 AA0105
# AA0056 5171-5174 (WT / syn cre relevant)

In [ ]:
# df.query('Cage.str.contains("AA0105|AA0056")')[easyCats].to_clipboard(index = False, sep = "\t")

In [ ]:
# df.query('Cage.str.contains("AA0105|AA0056")')[['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'DOB', 'Lineage']].to_clipboard(index = False, sep = "\t")

In [ ]:
# mice = df.fillna("-").query('Tag.str.contains("5049|5253|5254|5242")')
# cages = mice.Cage.unique()

In [ ]:
# # cages = mice.Cage.unique()
# df[df.Cage.isin(cages)][easyCats].to_clipboard(index = False, sep = "\t")

In [ ]:
# df[df.Cage.isin(cages)][['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'DOB', 'Lineage']]

# code from summing total hgRNA counts for collecting animal samples

## splitting hgRNA genotypes into columns

In [ ]:
## speciality speed code for re-splitting genotypes by hgRNA tyep
def splitGenotypes(df):
    '''for splitting genotypes into hgRNA categories and summing to pick breeders when Reza hasn't, or for distribution, or whatever'''
    split = df['Genotype'].str.split(r'(\w,)', expand=True)
    split[6] = split[6].str.extract('(\d+)')
    split = split.rename({
        0: "E",
        2: 'I',
        4: "S",
        6: 'ina'
    }, axis=1).drop(columns=[1, 3, 5])

    newcols = []
    oldcols = []
    for col in split:
        colN = str(col) + "_num"
        split[colN] = pd.to_numeric(split[col], errors='coerce')
        oldcols.append(col)
        newcols.append(colN)

    split['summedhgRNAs'] = split[newcols].sum(axis=1)
    split['summedhgRNAs_active'] = split[newcols[0:3]].sum(axis=1)
    split = split.drop(oldcols[0:4], axis=1)
    df = df.merge(split, left_index=True, right_index=True)
    return df

In [ ]:
df = splitGenotypes(df)

In [ ]:
breedersMask = df['Tag'].isin(set(breeders['Tag']))
euthanasiaMask = (df['DOD'] == 'requested euthanasia')
fdMask = df.DOD.str.contains("fd|breeder-may|experimental")
pbMask = df.Lineage.str.contains("PB#")
fMask = (df.Sex.str.lower() == "f")

In [ ]:
df[~breedersMask & ~euthanasiaMask & ~fdMask & pbMask & ~fMask].sort_values(['E_num', 'I_num', 'S_num'])[easyCats]

In [ ]:
# .to_clipboard(index=False, sep = "\t")
df.query('DOD.str.contains("zack")')[['Tag','Genotype', 'summedhgRNAs', 'summedhgRNAs_active']].to_csv('ZackTransferGenotypes.tsv', sep = '\t', index=False) 

## query each lineage + create CSV from it

# checking breedings against spreadsheet

In [ ]:
source_dir = "Breedings"
date = "2020-10-09"
fns = glob.glob(f"{source_dir}/*{date}_cage*")
data = []

for fn in fns:
    frame = pd.read_csv(fn, sep="\t")
    frame['Lineage'], frame['Date'] = fn[10:24].split("-", 1)
    data.append(frame)

animals = pd.concat(
    data, ignore_index=True)  #dont want pandas to try an align row indexes

animals = animals.applymap(lambda x: x.strip("t") if isinstance(
    x, str) else x)  #remove t for easier comparison to big frame applys

tagCols = ['Male', 'Female1',
           'Female2']  #grab columns with animals for flattening

data = []
for col in tagCols:
    frame = pd.concat((animals[i] for i in [col, 'Lineage', 'Date']),
                      axis=1).rename({col: 'Tag'}, axis=1)
    data.append(frame)

#            #, animals['Lineage'], animals['Date']]
# #     data.append(frame)
breeders = pd.concat(
    data, ignore_index=True)  #dont want pandas to try an align row indexes;

## check that all animals are correctly marked in the spreadsheet

In [ ]:
## #### parsing breedings etc - check that all breeders are marked as such on the sheet
breedrs = pd.merge(df, breeders, left_on ='TagClean', right_on = 'Tag', how = 'right')
if len(breedrs.DOD.unique()) > 1:
    print("Not all breeders marked")
else:
    print("Only one DOD found, {}".format(breedrs.DOD.unique()[0]))
    
list(set(breeders['Tag'])-set(breedrs['TagClean']))

In [ ]:
breedrs.query('DOD.str.contains("pbfbreeder_tier1", na = False)').groupby("Cage").agg({'Cage': 'nunique', 'TagClean': ['nunique','unique'], 'Lineage_x':'unique'})

# something eles happens here? something about querying for samples of a specific type

In [ ]:
# def query(df, lineage, disp=1):
#     result = df.query(
#         'Lineage.str.contains("{}")'.format(lineage))
#     if disp:
#         result = result.query('DispPercent >= {}'.format(disp))
#     result = result.sort_values("summedhgRNAs_active", ascending=False)
#     return result

# frames = {}
# lineages = ['#3', '#7', 'ins21', '-Cas9', 'PBF']

# for line in lineages:
#     frame = query(df, line).head(5)
#     frames.update({line: frame})
# for frame in frames:
#     fn = makefn(frame)
#     frames[frame].to_csv(fn, sep="\t")
#     df[['fCas', 'transgene']] = df['Genotype'].str.split(",", 1, expand = True)
# frame = query(df, 'fCas9').query('transgene.str.contains("-")')
# frame = frame.sort_values(['fCas', 'transgene'], ascending = [True, True]).head(10)
# fn = makefn('creFlipped-negative_ampliconSamples')
# frame.to_csv(fn, sep ='\t')


In [ ]:
##############################################

# assessing parental genotypes to consider offspring (for fCas9 maintenance/breeding)

In [ ]:
df.Lineage = df.Lineage.fillna("--")
fathers = df[df.Lineage.str.contains('fCas9')].Father.unique().flatten()
mothers = df[df.Lineage.str.contains('fCas9')].Mother.unique().flatten()
rents = np.append(fathers, mothers)
parentList = []
for i in rents:
    parentList.extend(i.split("/")) 
parents = df.loc[df['Tag'].isin(parentList)]

In [ ]:
## easy categories
plateCats =  ['Genotype','PlateID', 'Row', 'Column',
       'Position']
cats = easyCats + plateCats

In [ ]:
df[['fCas', 'transgene']] = df['Genotype'].str.split(",", 1, expand = True)

In [ ]:
df.Lineage = df.Lineage.fillna("--")
linMask = df.Lineage.str.contains('fCas9')
genoMask = df.Genotype.isnull()
fCas9 = df[linMask].fillna("-")
H_linMask = fCas9.Lineage.str.contains('hCD2')
S_linMask = fCas9.Lineage.str.lower().str.contains('syn')

In [ ]:
df.query("Tag == '5326'")

In [ ]:
fCas9.groupby(['Sex', 'DOD', 'fCas', 'transgene']).agg({'Genotype': 'count',
                                                   'Tag':'unique', 'Cage':'unique', 
                                                  'DOB': ['min', 'max']})

In [ ]:
cages = df.groupby(["Cage", "Sex"]).agg({'Color':'unique',
                                         'DOD':'unique',
                                         'Lineage':'unique'})   #yay!

In [ ]:
df.groupby('Cage').agg({
    'dispensible': 'mean',
    'Cage': 'size',
    'Sex': 'unique',
    'DOD': 'unique',
    'Tag': 'unique',
    'Color': 'unique',
    'Ear': 'unique',
    'Lineage': 'unique'
}).sort_values('dispensible', ascending = False)

In [ ]:
df[df.Sex == "F"].groupby(['Cage']).agg({'Cage': 'size',
                                         'DOD': 'unique',
                                         'Tag':'unique',
                                        'Color' : 'unique',
                                         'Ear' : 'unique',}).sort_values

In [ ]:
df[df.Sex == "F"].groupby(['Cage']).agg({'Cage':'size',
                                         'Color':'unique',
                                         'DOD':'unique',
                                         'Lineage':'unique'})   #yay!

# retrieve animals with null genotypes in a certain lineage + get their well plates

In [ ]:
insGeno = df[(df.Lineage.str.contains("ins21")) & (df.Genotype.isnull())]

In [ ]:
locs = pd.read_pickle('96-wellLocs.pkl').sort_values(
        ['Column',
         'Row']).reset_index(drop=True)  # impt standard plate indexing
# locs.join(df, rsuffix = "_init")
# insGeno.sort_values(join(locs, rsuffix = 'init')
locs.join(insGeno, rsuffix='init')

In [ ]:
insGeno.pivot(index='WellRow', columns='WellCol',
                      values=['Tag', 'Plate'])

# generate a general colony pivot table by lineage, sex, and DOB with total numbers

In [ ]:
file = "05227-mst.tsv"
saveFn = makefn("05227-ms".replace(" ", "")+'_summary')
remapDict= {'ins': '1685ins21#2',
            'pb3' : '1763PB#3',
            'pb7' : '1763PB#7',
            'pbf': '1853PBF#1'}
for key, value in remapDict.items():
    df.Lineage = df.Lineage.apply(lambda row: value if key in row.lower() else row)
grouped = df.groupby(['Lineage',
            'Sex',
            pd.Grouper(key='DOB',freq='q')]).size().unstack('DOB').reset_index()
grouped['Total'] = grouped.sum(axis=1)
grouped.fillna("-").to_csv(saveFn)

In [ ]:
saveFn = makefn(file.replace(" ", "")+'_femaleConsolidation')

In [ ]:
df.groupby(["Cage"]).agg({'Cage': 'count',  'Sex': 'unique',
                                 'Color':'unique',
                                'DOD': 'unique'})

In [ ]:
mating = tagSearch(ins + PB3 + PB7 + PBF + cre)
df[mating].sort_values('Lineage').to_csv('file.csv')

# break - 2019 and previous code snippets

In [ ]:
CreMask = (df.Lineage.str.contains("fCas", na=False))
PBFMask = (df.Lineage.str.contains("PBF", na=False))
insMask = tagSearch("t2620 t2632 t2623 t2463 t2625 t2622 t2616 t2626 t2633 t2720 t2721 t2722 t2755 t2750 t2751 t2415 t2445")
alreadyShipped = (df.Cage.str.contains("HOP", na= True))
traveling = (df.DOD.str.endswith("hopkins| ", na = False))

In [ ]:
shipped_2 = df[(df.DOD.str.endswith("hopkins", na = False))]

In [ ]:
shipped_2.sort_values(["Cage", "Tag"])[['Cage', 'Sex', 'Tag', 'Ear', 'Color', 'DOB']].to_clipboard(excel=True, index=False)

In [ ]:
shipped_2.groupby(["Cage"]).agg({'Cage': 'count', 'Sex':'unique'}).to_clipboard(excel=True)

In [ ]:
hop = df[((traveling & (CreMask | PBFMask)) | insMask ) & ~alreadyShipped]

In [ ]:
hop.groupby(["Cage", "Sex"]).agg({'Cage': 'count'})

In [ ]:
hop.groupby(["Sex", "Cage", "Tag"]).agg({'Lineage': 'unique',
                                         'Color': 'unique',
                                        'Ear': 'unique'})

In [ ]:
hop.groupby(['Sex','Cage']).agg({'Cage':'count'})

In [ ]:
hop.sort_values(['Sex', "Lineage", 'Tag'])

In [ ]:
df[(df.Genotype.isnull()) \
   & (df['DOD'] != "Dispensible")].fillna("x").groupby(["Lineage", 'Cage']).agg({'Sex': 'count',
                                                                                   'Tag':'unique',
                                                                                   'Name': 'unique' })

In [ ]:
#mask for sorting centrifuge tube samples
freezerMask = (df.Tag.notnull()) & \
    (~df['Notebook info'].str.startswith("T|Plate", na=False))&\
     (df.Lineage != 'J024858-Cas9')

#and the results
df[freezerMask].sort_values("Tag")

In [ ]:
earlyMask, lateMask, fMask = collectAgeRange(df, '2019-12-01',
                                             28, 0 , "", "F")
earlyMask, lateMask, mMask = collectAgeRange(df, '2019-12-01',
                                             40, 0 , "", "M")

inc = df[mMask].append(df[fMask]) #combine

inc['DOD'], inc['parsedDOD'] =  zip(*inc.DOD.apply(lambda row: cleanDOD(row))) #parse DOD
inc = inc[(~inc.parsedDOD.notnull()) \
          & (~inc.DOD.str.contains('sperm|Diss|f.d.',
                                   na = False))] #drop assorted dead mice

In [ ]:
inc.fillna('n/a').groupby(['Lineage',
                           'DOD',
                           'Sex']).agg({'DOB': ['count',
                                                'min',
                                                'max']})

In [ ]:
inc[inc.DOD.str.contains('Hopkins', na= False)].groupby("Sex").agg({'Sex': 'count',
                                                    'Cage': 'nunique'})


In [ ]:
noMice, noCages

In [ ]:
df[df.Plate == "T885765"].sort_values(['WellRow',
                                       'WellCol'])[['Position',
                                                    'Tag',
                                                    'Lineage']].to_clipboard(excel=True, header = False, index = False)

In [ ]:
def is_coherent(seq):
    return seq == range(seq[0], seq[-1]+1)

In [ ]:
MMRRC_cats = ['Tag',
           'Cage',
           'Ear',
           'Sex',
           'Color',
           'Genotype',
           'DOB',
           'Lineage']

In [ ]:
# earlyMask, lateMask, mMask = collectAgeRange(df,
#                                              '2019-09-16',
#                                              13, 8,
#                                              "PB#", "M")
# earlyMask, lateMask, fMask = collectAgeRange(df, 
#                                              '2019-09-16',
#                                              4, 1,
#                                              "PB#", "F")
# shippedMask = (df.DOD.str.contains("mrrc|opkins",
#                                    na=False)) & (df.Lineage.str.contains("PB#")) #PBs marked for any maintenance movement

# df[mMask|fMask|shippedMask].groupby(["Lineage",
#                                      "Sex",
#                                      "DOD"]).agg({'Tag':'count',
#                                                    'DOB': ['first', 'last']}) #

dispense, cSorted = makeOutputs(df)
cSorted.to_clipboard(excel=True, index = True)

In [ ]:
earlyMask, lateMask, incMask = collectAgeRange(df, '2019-09-16', 4, 1, "", "F")
df[incMask][['Tag',
             'Cage',
             'Ear',
             'Sex',
             'Color',
             'DOB',
             'Genotype',
             'Lineage']].groupby("Lineage").count()

In [ ]:
earlyMask, lateMask, incMask = collectAgeRange(df, '2019-10-04', 10, 0 , "", "")
#inc = df[incMask]
#inc = inc.fillna("BLANK")

# [['Tag',
#              'Cage',
#              'Ear',
#              'Sex',
#              'Color',
#              'DOB',
#              'Genotype',
#              'Lineage']]

In [ ]:
# df[df.DOD == 'Shipped to mmrrc on 9/10'][['Tag',
#              'Color',
#              'Ear',
#              'Cage', 
#              'Sex',
#              'Genotype',
#              'DOB',
#              'Lineage']].sort_values(by = ['Cage',
#                                            'Tag'])\
#                 .fillna("missing").to_clipboard(index = False,
#                                                excel = True)

In [ ]:
# ## MICE WITH UNKNOWN FATE
# df[(df.DOD.isnull())].groupby(["Lineage", "Cage"]).agg({'DOB': ['max', 
#                                                              'min', 
#                                                              'count'],
#                                                         'Sex': 'max',
#                                                         'Name': 'unique'}).fillna("")

In [ ]:
########## selecting mice by age#########
shipDate = '2019-09-16'
early1, late1, mask1 = collectAgeRange(df, shipDate, 13, 8, "PB#", "M")
early2, late2, mask2 = collectAgeRange(df, shipDate, 9, 6.5, "PB#", "F")

males_20190916  = df[mask1].sort_values(["Lineage", "Genotype"])
females_20190916 = df[mask2].sort_values(["Lineage", "Genotype"])

shipment_20190916 = males_20190916.append(females_20190916)

# Sep09_shipment.sort_values(["Lineage",
#                             "Sex",
#                             "Cage"]).to_clipboard(excel=True,
#                                                   index = False)

# Sep09_shipment.groupby(["Lineage",
#                         "Sex",
#                         "Cage"])['Tag',
#                                  'Genotype'].count().to_clipboard()

In [ ]:
plate_reseq.sort_values(["Tag"])[['Tag',
                                  'Lineage',
                                  'Plate',
                                  'WellRow',
                                  'WellCol']].to_clipboard(excel = True, index= False)

In [ ]:
df['Tag'] = 't' + df['Tag'].astype(str)

In [ ]:
KL14_plateB = df[df['Plate'].str\
   .contains("T885773|T885761", na= False)]\
    .sort_values(by=['Plate', 'WellRow', 'WellCol']).reset_index(drop = True)

In [ ]:
KL14_plateB[['Tag','Lineage','Plate', 'WellRow', 'WellCol']]\
    .to_clipboard(excel=True, index=False)

In [ ]:
KL14_plateA = df[df['Plate'].str\
   .contains("T885772", na= False)]\
    .sort_values(by=['Plate', 'WellRow', 'WellCol']).reset_index(drop = True)

In [ ]:
df[df['Plate'].str\
   .contains("T88572|T885761|T885772|T885773",
             na = False)][['Tag', 'Plate', 'Well']]\
    .sort_values(by=['Plate', 'Well'])\
    .to_clipboard(excel=True, index= False)

In [ ]:
KL14_plateA[['Tag','Lineage','Plate', 'WellRow', 'WellCol']]\
    .to_clipboard(excel=True, index=False)

In [ ]:
#KL14_plateA.reindex('WellCol', method='ffill')

In [ ]:
KL14_plateA.groupby('WellRow').Lineage.count()

In [ ]:
KL14_plateA

In [ ]:
KL14_plateA[['Tag', 'Plate', 'WellRow', 'WellCol']]\
    .sort_values(by=['Plate', 'WellRow', 'WellCol'])\
    .to_clipboard(excel=True, index= False)

In [ ]:
## lineage stats over time
df.groupby([pd.Grouper(key="DOB", freq = "M"),
            "Lineage"]).Cage.count().unstack('DOB')

In [ ]:
def dfContains(string, column = "Cage", keep_empties = False, case = False, df = df):
    '''given a cage number/s (formatted as "nrbxxxx|nrbxxx") etc, the indexes of entries where true'''
    return df[column].str.contains(string, na = keep_empties, case = case)

In [ ]:
df[df.DOD.str.contains("Sperm", na = False)].groupby('Lineage').count()

In [ ]:
df[(dfContains("1036|1037"))
   & (df.Sex == "M")][['Tag',
                      'Cage',
                       'Ear',
                       'Sex',
                       'DOB',
                       'Notes']]

In [ ]:
PB = ['1763PB#7', '1763PB#3']
counted = df.groupby("Cage").size().reset_index(name = 'Total')
df = df.merge(counted)
PB_solo_males = df[(df.Lineage.isin(PB))&(df.Sex == "M")&(df.Total == 1)]